In [3]:
from  transformers import  *

2022-09-15 06:53:28.112788: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-15 06:53:28.112837: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# enocode  audion file  to  features
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading: 100%|██████████| 163/163 [00:00<00:00, 79.9kB/s]
Downloading: 100%|██████████| 85.0/85.0 [00:00<00:00, 40.1kB/s]
loading file vocab.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/vocab.json
loading file tokenizer_config.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/tokenizer_config.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/special_tokens_map.json
Downloading: 100%|██████████| 1.60k/1.60k [00:00<00:00, 629kB/s]
loading configuration file config.json from cache at /home/codespace/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/706111756296bc76512407a11e69526cf4e22aae/con